# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
weather_data= pd.read_csv('weather_data.csv', encoding="utf-8")
weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.12,-134.97,74.86,83,98,12.50,PF,1606267200
1,1,Goderich,43.75,-81.72,34.00,85,100,20.74,CA,1606267307
2,2,Cabadiangan,9.75,122.47,83.10,72,95,14.00,PH,1606267454
3,3,Nouadhibou,20.93,-17.03,71.60,46,87,9.17,MR,1606267264
4,4,Saldanha,-33.01,17.94,53.60,76,83,1.12,ZA,1606267455


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
#Store Lat & Lng Locations
locations= weather_data[["Lat", "Lng"]]
humidity= weather_data["Humidity"].astype(float)

In [24]:
#Plotting heat map with layer

fig = gmaps.figure()

heat_map = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=1)

fig.add_layer(heat_map)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
my_cities= weather_data.loc[(weather_data["Max Temp"]>67) & (weather_data["Max Temp"]<82) & (weather_data["Humidity"]<40) & (weather_data["Humidity"]>28) & (weather_data["Cloudiness"]==0), :]
my_cities= my_cities.dropna(how='any')
my_cities.reset_index(inplace=True)
del my_cities['index']
my_cities

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,34,Mataura,-46.19,168.86,70.21,36,0,5.06,NZ,1606267218
1,82,Wau,7.70,28.00,75.58,38,0,2.91,SS,1606267473
2,328,Abū Zabad,12.35,29.25,71.55,29,0,10.16,SD,1606267536
3,335,Port Lincoln,-34.73,135.87,79.48,33,0,7.61,AU,1606267538
4,526,Ouadda,8.08,22.40,69.94,39,0,3.56,CF,1606267585


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
hotel_df=[]

for i in range(len(my_cities)):
    lat= my_cities.loc[i]['Lat']
    lng= my_cities.loc[i]['Lng']
    
    target_type = "hotel"
    target_radius = 5000
    
    params = {
    "location": f"{lat}, {lng}",
    "type": target_type,
    "radius": target_radius,
    "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    
    cities_data = response.json()

#print(json.dumps(cities_data["results"][0], indent=4, sort_keys=True))
#results [name, location (lat/lng), vicinity]
    
    for x in cities_data["results"][0]:
        hotel_df.append(json.dumps(x))

my_cities["Hotel"]= hotel_df
my_cities

ValueError: Length of values does not match length of index

In [ ]:
#Store 1st hotel result into the DataFrame

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
